## Imports

In [ ]:
# !pip3 install os
from os import listdir
import string
from pickle import dump,load

## Loading the Data

In [ ]:
class LoadData:
    def __init__(self, directory):
        self.directory= directory
        
    def load_stories(self):
        """
        Load the data and store it in a list of dictionaries
        
        """
        all_stories= list()
        
        def load_doc(filename):
            """
            Return the data from a given filename
            """
            file = open(filename, encoding='utf-8')
            text = file.read()
            file.close()
            return text
        
        def split_story(doc):
            """
            Split story from summaries based on the separater -> "@highlight"
            """
            index = doc.find('@highlight')
            story, highlights = doc[:index], doc[index:].split('@highlight')
            highlights = [h.strip() for h in highlights if len(h) > 0]
            return story, highlights
        
        list_of_files= listdir(self.directory)
        for name in list_of_files[:1000]:
            filename = self.directory + '/' + name
            doc = load_doc(filename)
            story, highlights= split_story(doc)
            all_stories.append({'story': story, 'highlights': highlights})
        
        return all_stories

In [ ]:
DIR_PATH= "/home/nikhil/Downloads/cnn/stories"
obj= LoadData(DIR_PATH)
stories= obj.load_stories()

In [ ]:
len(stories)

In [ ]:
print(stories[10]['highlights'])
print()
print(stories[10]['story'])

In [ ]:
stories[:2]

## Data Cleaning

In [ ]:
class Clean_data:
    def __init__(self):
        pass
           
    def clean_lines(self, lines):
        cleaned = list()
        table = str.maketrans('', '', string.punctuation)
        
        for line in lines:
            index = line.find('(CNN)')
            if index >= 0:
                line = line[index + len('(CNN)'):]

            split_line = line.split()
            
            split_line = [word.lower() for word in split_line]
            split_line = [w.translate(table) for w in split_line]
            
            split_line = [word for word in split_line if word.isalpha()]
            cleaned.append(' '.join(split_line))
        cleaned = [c for c in cleaned if len(c) > 0]
        return cleaned

In [ ]:
obj1= Clean_data()
cleaned_stories= list()
for example in stories[:100]:
    cleaned_stories.append({'story': obj1.clean_lines(example['story'].split('\n')), 'highlights': obj1.clean_lines(example['highlights'])})    

In [ ]:
cleaned_stories[60]

In [ ]:
dump(cleaned_stories, open('/home/nikhil/Downloads/cnn/processed_sample_data/cnn_dataset.pkl', 'wb'))

In [ ]:
cleaned_stories = load(open('/home/nikhil/Downloads/cnn/processed_sample_data/cnn_dataset.pkl', 'rb'))
print('Loaded Stories %d' % len(cleaned_stories))

---

## Amazon Food reviews Dataset

## Imports

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
AMAZON_DATA_PATH= '/home/nikhil/Downloads/amazon-fine-food-reviews/Reviews.csv'

In [ ]:
class Load_amazon_data:
    def __init__(self, dir_path, seed= 0):
        self.dir_path= dir_path
        np.random.seed(seed)
        
    def load(self):
        """
        Reads data from the given directory path
        """
        return pd.read_csv(self.dir_path)
    
    def drop(self):
        """
        Drops unnecessary columns
        """
        
        data= self.load()
        
        data = data.dropna()
        data= data.iloc[:, -2:]
        data = data.reset_index(drop= True)
        
        return data
    
    def analyze_data(self):
        """
        Prints some sample data points from the cleaned data
        """
        data= self.drop()
        
        for sr_no, i in enumerate(np.random.randint(10, 100, size= 5)):
            print("_________________________")
            print("Data Point {0}".format(sr_no + 1))
            print("Summary:")
            print(data['Summary'].iloc[i])
            print("Full Text:")
            print(data['Text'].iloc[i])

In [ ]:
obj= Load_amazon_data(AMAZON_DATA_PATH, seed= 1)

## Load the Data

In [ ]:
data= obj.load()
data.head()

## Dropping Unnecessary columns

In [ ]:
data= obj.drop()
data.head()

## Analyze the data

In [ ]:
obj.analyze_data()

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
class Data_cleaning:
    def __init__(self):
        self.clean_summaries= []
        self.clean_texts= []

    def clean_text(self, text, remove_stopwords = False):
        """
        Defines a series of cleaning operations 
        """
        text = text.lower()

        if True:
            text = text.split()
            new_text = []
            for word in text:
                if word in contractions:
                    new_text.append(contractions[word])
                else:
                    new_text.append(word)
            text = " ".join(new_text)

        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text) 
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'<br >', ' ', text)
        text = re.sub(r'<br  >', ' ', text)
        text = re.sub(r'\'', ' ', text)

        # Optionally, remove stop words
        if remove_stopwords:
            text = text.split()
            stops = set(stopwords.words("english"))
            text = [w for w in text if not w in stops]
            text = " ".join(text)

        return text
    
    def clean(self, data):
        """
        Applies the clean_text() to the entire dataset
        """
        for summary in data.Summary:
            self.clean_summaries.append(self.clean_text(summary))

        print("Summaries are complete.")

        for text in data.Text:
            self.clean_texts.append(self.clean_text(text))

        print("Texts are complete.")
        
        return self.clean_summaries, self.clean_texts

In [ ]:
# import nltk
# nltk.download('stopwords')

clean_obj= Data_cleaning()
clean_summaries, clean_texts= clean_obj.clean(data)

In [ ]:
np.random.seed(1)

for sr_no, i in enumerate(np.random.randint(10, 100, size= 5)):
    print("_________________________")
    print("Data Point #{0}".format(sr_no + 1))
    print("Summary:")
    print(clean_summaries[i])
    print("Full Text:")
    print(clean_texts[i])